# Setup & Test Access to Azure Data Lake Storage Gen2

**Purpose:** 
- Test connection to ADLS Gen2
- Mount storage containers (bronze, silver, gold)
- Verify data access

**Last Updated:** Feb 9, 2026  
**Author:** Kevin


## Configuration

Define storage account details


In [0]:
storage_account_name = "stgolistmigration"

account_key = ""

# Configure Spark auth
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    account_key
)

print("✅ Auth configured")

✅ Auth configured


## Helper Functions

In [0]:
def get_bronze_path(folder, filename):
    """Bronze layer path builder"""
    return f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/olist/{folder}/{filename}"

def get_silver_path(table):
    """Silver layer path builder"""
    return f"abfss://silver@{storage_account_name}.dfs.core.windows.net/{table}/"

def get_gold_path(table):
    """Gold layer path builder"""
    return f"abfss://gold@{storage_account_name}.dfs.core.windows.net/{table}/"

print("✅ Path helpers ready")

✅ Path helpers ready


## Verify Bronze

In [0]:
print("Checking bronze data...")

base_path = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/olist"

data_files = {
    "Orders": f"{base_path}/orders/oracle_OLIST.OLIST_ORDERS_BASE.parquet",
    "Order Items": f"{base_path}/orders/OLIST.OLIST_ORDER_ITEMS_BASE.parquet",
    "Customers": f"{base_path}/customers/OLIST.OLIST_CUSTOMERS_BASE.parquet",
    "Payments": f"{base_path}/payments/OLIST.OLIST_ORDER_PAYMENTS_BASE.parquet"
}

total = 0
for name, path in data_files.items():
    try:
        df = spark.read.parquet(path)
        rows = df.count()
        cols = len(df.columns)
        total += rows
        print(f"✅ {name:15} {rows:>8,} rows | {cols:>2} cols")
    except Exception as e:
        print(f"❌ {name:15} Error: {str(e)[:60]}")

print(f"\nTotal: {total:,} records")
print("Setup complete - ready for transformations")

Checking bronze data...
✅ Orders            99,441 rows |  8 cols
✅ Order Items      112,650 rows |  7 cols
✅ Customers         99,441 rows |  5 cols
✅ Payments         103,886 rows |  5 cols

Total: 415,418 records
Setup complete - ready for transformations
